## 1. Формулировка

#### 1.1 Описание задачи

Named Entity Recognition (распознавание именованных сущностей) — это задача автоматического выявления и классификации сущностей в тексте.

В данной задаче нужно выделить и классифицировать следующие классы сущностей:

- PER — персоналии (люди)
- ORG — организации
- LOC — географические объекты
- EVT — события
- PRO — процессы

**Пример:**

> "Тереза Мэй выступила в Парламенте Великобритании после саммита ЕС."

```python

entities = [
    ("Тереза Мэй", "PER"),
    ("Парламент Великобритании", "ORG"),
    ("ЕС", "ORG"),
    ("саммит", "EVT")
]
```

#### 1.2 Классические Методы

1. **Rule-based**

Написание шаблонов/паттернов и/или составление списков сущностей.

2. **Машинное Обучение**

- HMM (Hidden Markov Model) - татистическая модель, используемая для анализа последовательностей данных, где наблюдаемые события зависят от скрытых.
- CRF (Conditional Random Fields) - использует случайно заведённые признаки слов/токенов. Например, морфология слов, регистр, пунктуция, POS-теги или приндлежность к списку других сущностей

3. **Глубокое Обучение**

- BiLSTM (Biderectional Long-Short Term Memory) - достаточна легвоесная архитектура, для разного рода задач над последовательностями.

- BERT (Biderctional Encoder Representations from Transformers) - более весомая архитектура, обращающая внимание с обеих сторон и автоматически извлекающая признаки (не интрепретируемые в большинстве своём).

#### 1.3 Решение через LLM

Подход через prompt engineering:

1. Prompt "Извлеки все сущности из следующего текста. Укажи тип (PER, ORG, LOC, EVT, PRO) для каждой сущности."
2. Загружаем в модель prompt + текст
3. Получаем структурированный ответ (например в json)

#### 1.4 Оценка качества

Имеем тестовый датасет, который не попадал в трейн.

По нему можно собрать классические метрики:

$$
Precision = \frac{TP}{TP + FP}
$$

$$
Recall = \frac{TP}{TP + FN}
$$

$$
F_1 = 2 * \frac{Precision * Recall}{Precision + Recall}
$$

Также можем выделить разные уровни агрегации:

- По документам
- По типу сущностей
- Общая макро/микро усреднённая метрика (зависит от дизбаланса присутсвия классов)


## 2. Чтение


In [13]:
import pandas as pd
from pathlib import Path


def get_dataframe(dataset_path: Path) -> pd.DataFrame:
    data = []

    raw_files = sorted((dataset_path / "raw" / "ru").glob("*.txt"))
    for raw_file in raw_files:
        with open(raw_file, encoding="utf-8") as f:
            lines = [line.strip() for line in f.readlines()]
            if len(lines) >= 5:
                data.append(
                    {
                        "document_id": lines[0],
                        "language": lines[1],
                        "creation_date": lines[2],
                        "url": lines[3],
                        "title": lines[4],
                        "document_text": "\n".join(lines[5:]),
                    }
                )

    df = pd.DataFrame(data)

    # Entities and gold answers
    entity_data = []
    gold_data = []

    annotated_files = sorted((dataset_path / "annotated" / "ru").glob("*.out"))
    for annotated_file in annotated_files:
        entities = []
        with open(annotated_file, encoding="utf-8") as f:
            lines = [line.strip() for line in f.readlines()]
            # Skip the first line (document_id)
            for line in lines[1:]:
                if line:
                    parts = line.split("\t")
                    if len(parts) >= 3:
                        mention, base_form, category = parts[:3]
                        entities.append(
                            {
                                "mention": mention,
                                "base_form": base_form,
                                "category": category,
                            }
                        )
        if entities:
            entity_data.append([e["mention"] for e in entities])
            gold_data.append([e["category"] for e in entities])
        else:
            entity_data.append([])
            gold_data.append([])

    df["entity"] = entity_data
    df["gold_answer"] = gold_data

    return df

In [14]:
dataset_path = Path("sample_pl_cs_ru_bg")
df = get_dataframe(dataset_path)
df

,document_id,language,creation_date,url,title,document_text,entity,gold_answer
0,ru-10,ru,2018-09-20,https://rg.ru/2018/09/20/tereza-mej-rasschityv...,Тереза Мэй рассчитывает усидеть в седле до зав...,"\nТем не менее, по сведениям британских СМИ, н...","[Brexit, Альбиона, Альбионе, Борис Джонсон, Бр...","[EVT, LOC, LOC, PER, LOC, LOC, LOC, LOC, LOC, ..."
1,ru-1000,ru,2018-07-09,http://news.meta.ua/ua/cluster:60407124-Boris-...,Подробности.ua: Третий за сутки: Борис Джонсон...,"\nТаким образом, Джонсон стал третьим министро...","[Brexit, The Guardian, Борис Джонсон, Бориса Д...","[EVT, PRO, PER, PER, PER, LOC, PER, PER, ORG, ..."
2,ru-1001,ru,2018-07-09,https://rg.ru/2018/07/09/boris-dzhonson-podal-...,Глава МИД Британии Борис Джонсон подал в отставку,\nМинистр иностранных дел Великобритании Борис...,"[Associated Press, Brexit, Борис Джонсон, Бори...","[ORG, EVT, PER, PER, LOC, PER, PER, ORG, ORG, ..."
3,ru-1002,ru,2018-07-09,https://echo.msk.ru/news/2237252-echo.html,09.07.2018 18:09 : Премьер-министр Великобрита...,\nПремьер-министр Великобритании Тереза Мей пр...,"[Бориса Джонсона, Брекзит, Великобритании, Дэв...","[PER, EVT, LOC, PER, ORG, ORG, PER]"
4,ru-1003,ru,2018-07-09,https://echo.msk.ru/news/2237216-echo.html,09.07.2018 17:02 : Министр иностранных дел Вел...,\nОб этом сообщает агентство Рейтер. Сегодня п...,"[Борис Джонсон, Великобритании, Даунинг-Стрит,...","[PER, LOC, LOC, PER, PER, ORG, PER, ORG, PER, ..."
5,ru-1004,ru,2018-07-09,http://www.aif.ru/politics/world/dzhonson_ushe...,Борис Джонсон ушел в отставку с поста главы МИ...,\nГлава МИД Великобритании Борис Джонсон ушел ...,"[Brexit, Борис Джонсон, Германии, Джонсон, Джо...","[EVT, PER, LOC, PER, PER, PER, PER, ORG, ORG, ..."
6,ru-1006,ru,2018-07-09,http://polit.ru/news/2018/07/09/zakharova/,Захарова лирически прокомментировала отставку ...,\nОфициальный представитель МИД России Мария З...,"[Brexit, Facebook, Борис Джонсон, Бориса Джонс...","[EVT, PRO, PER, PER, LOC, LOC, PER, PER, PER, ..."
7,ru-1011,ru,2018-07-09,https://tsargrad.tv/news/za-borisom-dzhonsonom...,За Борисом Джонсоном последует сама Мэй - брит...,\nКомментируя отставку министра иностранных де...,"[Brexit, The Guardian, Борис Джонсон, Бориса Д...","[EVT, PRO, PER, PER, PER, LOC, LOC, PER, PER, ..."
8,ru-1017,ru,2018-07-09,http://www.unn.com.ua/ru/news/1740316-boris-dz...,Борис Джонсон подал в отставку с поста министр...,\nКИЕВ. 9 июля. УНН. Борис Джонсон подал в отс...,"[Brexit, The Guardian, Борис Джонсон, Бориса Д...","[EVT, PRO, PER, PER, PER, PER, LOC, PER, PER, ..."


## 3. Входное для LLM


In [15]:
import yaml

with open("config.yaml") as config_file:
    config = yaml.safe_load(config_file)

for k, v in config.items():
    print(f"{k}: {len(v) * '*'}")

client_id: ************************************
scope: *****************
auth_key: ****************************************************************************************************
client_secret: ************************************


In [ ]:
from pathlib import Path
from langchain_gigachat.chat_models import GigaChat
from langchain_core.language_models import BaseChatModel
from langchain.prompts import load_prompt, BasePromptTemplate
# from langchain_core.messages import HumanMessage, SystemMessage

giga = GigaChat(
    credentials="",
    verify_ssl_certs=False,
    scope=config["scope"],
    model="GigaChat-Lite"
)

prompt = load_prompt("template.yaml")
print(prompt)

input_variables=['text'] input_types={} partial_variables={} template='Проанализируй следующий текст и извлеки все пары наименованных сущностей, относящихся к следующим классам:\n- PER — персоналии (люди)\n- ORG — организации\n- LOC — географические объекты\n- EVT — события\n- PRO — процессы\n\nПредставь результат в виде JSON, соответствующего следующей модели Pydantic:\n\n```python\nfrom pydantic import BaseModel, Field\nfrom typing import List\n\nclass Pair(BaseModel):\n    entity: str = Field(..., description="Наименованная сущность")\n    answer: str = Field(..., description="Класс сущности (PER, ORG, LOC, EVT, PRO)")\n\nclass Response(BaseModel):\n    pairs: List[Pair]\n```\n\nТекст для анализа:\n{text}\n'


In [ ]:
def ask_llm_for_ner(model: BaseChatModel, prompt: BasePromptTemplate, text: str):
    prompt = prompt.format(text=text)
    chain = prompt | model
    return chain  # .generate_text()